# Загрузка данных и создание обучающих датасетов

In [ ]:
import pandas as pd

In [ ]:
popular_to_bert = pd.read_csv('popular_to_bert.csv')
popular_to_bert.head(5)

,query,query_popularity,splited
0,куртка женская осенняя,10,3
1,ботинки женские,10,2
2,пальто женское осеннее,10,3
3,пальто женское,10,2
4,джинсы женские,10,2


In [ ]:
!pip install transformers
from transformers import BertTokenizer, BertForMaskedLM
import torch

name = 'cointegrated/rubert-tiny2' 

tokenizer = BertTokenizer.from_pretrained(name)
model = BertForMaskedLM.from_pretrained(name)
None

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def get_mask(x):
    x = x.split(' ')
    n_mask = len(tokenizer.tokenize(x[-1]))
    x.remove(x[-1])
    x +=  ['[MASK]'] * n_mask
    x = " ".join(x)
    return x

In [ ]:
get_mask('куртка женская осенняя')

'куртка женская [MASK] [MASK]'

In [ ]:
%%time
popular_to_bert['query_mask'] = popular_to_bert['query'].apply(lambda x: get_mask(x) )

CPU times: user 17.6 s, sys: 199 ms, total: 17.8 s
Wall time: 17.8 s


In [ ]:
popular_to_bert.head(5)

,query,query_popularity,splited,query_mask
0,куртка женская осенняя,10,3,куртка женская [MASK] [MASK]
1,ботинки женские,10,2,ботинки [MASK]
2,пальто женское осеннее,10,3,пальто женское [MASK] [MASK]
3,пальто женское,10,2,пальто [MASK]
4,джинсы женские,10,2,джинсы [MASK]


In [ ]:
list_popular = list(popular_to_bert['query'])
list_popular_mask = list(popular_to_bert['query_mask'])
print(list_popular[0],'\n', list_popular_mask[0]  )

куртка женская осенняя 
 куртка женская [MASK] [MASK]


In [ ]:
%%time
inputs = tokenizer(list_popular, return_tensors='pt', max_length=12, truncation=True, padding='max_length')
inputs_mask = tokenizer(list_popular_mask, return_tensors='pt', max_length=12, truncation=True, padding='max_length')

CPU times: user 1min 38s, sys: 985 ms, total: 1min 39s
Wall time: 1min 39s


In [ ]:
print(inputs['input_ids'][0],'\n', inputs_mask['input_ids'][0]  )

tensor([    2, 40769,   872, 50967, 44497,  9094,     3,     0,     0,     0,
            0,     0]) 
 tensor([    2, 40769,   872, 50967,     4,     4,     3,     0,     0,     0,
            0,     0])


In [ ]:
inputs['labels'] = inputs_mask.input_ids.detach().clone()

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
dataset = Dataset(inputs)

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=T

In [ ]:
312 * 83828

26154336

In [ ]:
all_sum = 0
for param in model.parameters():
    if param.requires_grad:
      all_sum +=param.numel()
all_sum

29278220

In [ ]:
#Заморозка параметров
# for param in model.bert.parameters():
#     param.requires_grad = False

# Заморозка всех параметров
for param in model.parameters():
    param.requires_grad = False

for name, param in model.cls.named_parameters():
    # print(name, type(name))
    if name == 'predictions.decoder.weight':
        param.requires_grad = True

In [ ]:
noall_sum = 0
for param in model.parameters():
    if param.requires_grad:
      noall_sum +=param.numel()
noall_sum

26154336

In [ ]:
noall_sum / all_sum * 100

89.3303486345823

In [ ]:
for param in model.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([83828, 312])


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
None

In [ ]:
# torch.cuda.empty_cache()
!nvidia-smi

Sat Nov 13 21:51:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    73W / 149W |    732MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=1e-5)

In [ ]:
from tqdm import tqdm  # for our progress bar

epochs = 1

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/2267 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """
Epoch 0: 100%|██████████| 2267/2267 [08:41<00:00,  4.34it/s, loss=3.9]


In [ ]:
model.eval()
model.to('cpu')
None

In [ ]:
def sim_word(text, tokenizer, N_tokens = 10):
    inputs = tokenizer(text, padding = True, truncation=True, return_tensors = 'pt')
    inp_tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    ind_mask = inp_tokens.index('[MASK]')
    outputs = model(**inputs)
    idx_sim_words = outputs.logits.argsort(dim = 2, descending=True)[0][ind_mask][:N_tokens]
    sim_tokens = tokenizer.convert_ids_to_tokens(idx_sim_words)
    return sim_tokens

In [ ]:
%%time
# text = 'Это миниатюрная модель для [MASK] разных задач.'

input_text = 'Пальто [MASK].'

print(input_text)
print(sim_word(input_text, tokenizer, N_tokens = 10))

Пальто [MASK].
['##чки', '[MASK]', '##лы', '##чка', '##ль', '##шка', '##ло', '##л', '##ик', '##чник']
CPU times: user 77.9 ms, sys: 967 µs, total: 78.9 ms
Wall time: 82 ms


In [ ]:
# model.save_predtrained('First_bert.bin')

In [ ]:
popular_to_bert

,query,query_popularity,splited,query_mask
0,куртка женская осенняя,10,3,куртка женская [MASK] [MASK]
1,ботинки женские,10,2,ботинки [MASK]
2,пальто женское осеннее,10,3,пальто женское [MASK] [MASK]
3,пальто женское,10,2,пальто [MASK]
4,джинсы женские,10,2,джинсы [MASK]
...,...,...,...,...
290140,уход за кожей,1,3,уход за [MASK]
290141,Темный чехол xr,1,3,Темный чехол [MASK] [MASK]
290142,конструктор робо-динозавры,1,2,конструктор [MASK] [MASK] [MASK] [MASK] [MASK]
290143,Дешевый товар,1,2,Дешевый [MASK]
